# AG2: GroupChat (Group Chat with Tools)

Group Chat with Tools
- https://docs.ag2.ai/docs/user-guide/advanced-concepts/groupchat/tools

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Group Chat with Tools

### Imports

In [2]:
import os
import random

from autogen import (
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    register_function,
    # LLMConfig,
)

### Agent Configuration

In [3]:
# llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"],)
llm_config = {"api_type": "openai", "model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}


sales_agent = ConversableAgent(
    name="SalesAgent",
    llm_config=llm_config,
)

cancellation_agent = ConversableAgent(
    name="CanelationAgent",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    code_execution_config={
        "use_docker": False,
    },  # Please set use_docker=True if Docker is available to run the generated code. Using Docker is safer and recommended over running the generated code directly on the host machine.
)

### Tools Registration

In [4]:
def buy_airplane_ticket(from_location: str, to_location: str, date: str) -> str:
    ticket_number = random.randint(1000, 9999)
    return f"""Your ticket from {from_location} to {to_location} on {date} has been booked.
Your ticket number is {ticket_number}.
Please keep this number for future reference.
"""

register_function(
    buy_airplane_ticket,
    caller=sales_agent,
    executor=user_proxy,
    description="Buy an airplane ticket",
)

def cancel_airplane_ticket(ticket_number: str) -> str:
    return f"Your ticket with ticket number {ticket_number} has been canceled"

register_function(
    cancel_airplane_ticket,
    caller=cancellation_agent,
    executor=user_proxy,
    description="Cancel an airplane ticket",
)

### Creating and Initiating the Group Chat

In [5]:
groupchat = GroupChat(
    agents=[user_proxy, cancellation_agent, sales_agent],
    speaker_selection_method="auto",
    messages=[],
)

manager = GroupChatManager(
    name="group_manager",
    groupchat=groupchat,
    llm_config=llm_config,
)

In [6]:
chat_result = user_proxy.initiate_chat(
    recipient=manager,
    message="I need to buy a plane ticket from New York to Los Angeles on 12th of April 2025",
)

user_proxy (to group_manager):

I need to buy a plane ticket from New York to Los Angeles on 12th of April 2025

--------------------------------------------------------------------------------

Next speaker: SalesAgent


>>>>>>>> USING AUTO REPLY...
SalesAgent (to group_manager):

***** Suggested tool call (call_48ZKPmC5WVDyYaWfvjCNN6uA): buy_airplane_ticket *****
Arguments: 
{"from_location":"New York","to_location":"Los Angeles","date":"2025-04-12"}
************************************************************************************

--------------------------------------------------------------------------------

Next speaker: user_proxy



Replying as user_proxy. Provide feedback to group_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [7]:
print(chat_result.summary)

None


In [8]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

2
[{'content': 'I need to buy a plane ticket from New York to Los Angeles on 12th of April 2025', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'None', 'tool_calls': [{'id': 'call_48ZKPmC5WVDyYaWfvjCNN6uA', 'function': {'arguments': '{"from_location":"New York","to_location":"Los Angeles","date":"2025-04-12"}', 'name': 'buy_airplane_ticket'}, 'type': 'function'}], 'name': 'SalesAgent', 'role': 'assistant'}]
